# 🚀 EnfoadsIA - Backend Production (T4 GPU)

Este notebook ejecuta el ecosistema completo de **EnfoadsIA** en Google Colab con GPU T4.

## 📋 Instrucciones
1. Asegúrate de tener GPU habilitada: `Runtime > Change runtime type > T4 GPU`
2. Ejecuta las celdas en orden
3. Copia la URL pública generada para conectar tu frontend

---

In [ ]:
# @title 🛠️ 1. Configuración del Entorno

import os
import sys
import subprocess
from pathlib import Path

# Configuración
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
BASE_DIR = Path("/content")
REPO_DIR = BASE_DIR / "FoadsIA"
BACKEND_DIR = REPO_DIR / "backend"

def run_command(cmd, description, silent=False):
    """Ejecuta un comando con manejo de errores."""
    print(f"⚙️  {description}...")
    try:
        if silent:
            result = subprocess.run(
                cmd, 
                shell=True, 
                check=True,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
        else:
            result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(f"✅ {description} completado")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Error en {description}: {e}")
        if not silent and hasattr(e, 'stderr'):
            print(f"   Detalles: {e.stderr[:200]}")
        return False

# Resetear al directorio base
os.chdir(BASE_DIR)
print(f"📁 Directorio base: {os.getcwd()}\n")

# Clonar o actualizar repositorio
print("📡 Configurando repositorio...")
if not REPO_DIR.exists():
    if run_command(f"git clone {REPO_URL}", "Clonando repositorio"):
        print("✅ Repositorio clonado exitosamente\n")
    else:
        raise Exception("No se pudo clonar el repositorio")
else:
    os.chdir(REPO_DIR)
    if run_command("git pull", "Actualizando repositorio"):
        print("✅ Repositorio actualizado\n")

# Verificar estructura del proyecto
if not BACKEND_DIR.exists():
    raise Exception(f"❌ No se encontró el directorio backend en {REPO_DIR}")

os.chdir(BACKEND_DIR)
print(f"✅ Directorio de trabajo: {os.getcwd()}\n")

# Instalar dependencias del sistema
print("📦 Instalando dependencias del sistema...")
system_packages = "ffmpeg libsm6 libxext6 libgl1"
run_command(
    f"apt-get update -qq && apt-get install -y -qq {system_packages}",
    "Dependencias del sistema",
    silent=True
)

# Instalar dependencias de Python
print("\n🐍 Instalando dependencias de Python...")
if (BACKEND_DIR / "requirements.txt").exists():
    run_command(
        "pip install -q -r requirements.txt",
        "Instalación de requirements.txt",
        silent=True
    )
else:
    print("⚠️  No se encontró requirements.txt")

# Asegurar dependencias críticas
print("\n🔧 Verificando dependencias críticas...")
critical_packages = "flask flask-socketio flask-cors eventlet pyngrok torch torchvision"
run_command(
    f"pip install -q {critical_packages}",
    "Dependencias críticas",
    silent=True
)

# Verificar GPU
print("\n🎮 Verificando disponibilidad de GPU...")
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU DETECTADA: {gpu_name}")
        print(f"   Memoria: {gpu_memory:.1f} GB")
        print(f"   CUDA Version: {torch.version.cuda}")
    else:
        print("⚠️  GPU no disponible - el modelo correrá en CPU (muy lento)")
except Exception as e:
    print(f"❌ Error al verificar GPU: {e}")

print("\n" + "="*60)
print("🎉 CONFIGURACIÓN COMPLETADA - Listo para ejecutar servidor")
print("="*60 + "\n")

In [ ]:
# @title 🚀 2. Ejecutar Servidor Backend

import os
import time
import threading
from pathlib import Path
from pyngrok import ngrok

# Token de ngrok (obtén el tuyo en https://dashboard.ngrok.com)
AUTH_TOKEN = "2yHQiBeYhFdbJSiK31054jtsKkw_54yvtD5Cs9mK2yhFgQ2j" #@param {type:"string"}
PORT = 5000 #@param {type:"integer"}

# Validar directorio
BACKEND_DIR = Path("/content/FoadsIA/backend")
if not BACKEND_DIR.exists():
    raise Exception(f"❌ Directorio backend no encontrado: {BACKEND_DIR}")

os.chdir(BACKEND_DIR)
print(f"📁 Directorio actual: {os.getcwd()}\n")

# Verificar app.py
if not (BACKEND_DIR / "app.py").exists():
    raise Exception("❌ No se encontró app.py en el directorio backend")

# Configurar ngrok
if AUTH_TOKEN:
    try:
        print("🔐 Configurando autenticación de ngrok...")
        ngrok.set_auth_token(AUTH_TOKEN)
        print("✅ Token configurado\n")
    except Exception as e:
        print(f"⚠️  Error al configurar token: {e}\n")
else:
    print("⚠️  No se proporcionó token de ngrok - la sesión será limitada\n")

# Limpiar túneles existentes
print("🧹 Limpiando túneles anteriores...")
try:
    ngrok.kill()
    time.sleep(1)
    print("✅ Túneles limpiados\n")
except Exception as e:
    print(f"ℹ️  No había túneles previos\n")

# Crear túnel
print(f"🌐 Creando túnel público en puerto {PORT}...")
try:
    tunnel = ngrok.connect(PORT, bind_tls=True)
    public_url = tunnel.public_url
    
    print("\n" + "="*70)
    print("🎉 SERVIDOR PÚBLICO ACTIVO")
    print("="*70)
    print(f"\n📡 URL del Backend: {public_url}")
    print(f"\n💡 Usa esta URL en tu frontend para conectarte al servidor")
    print(f"\n⚠️  IMPORTANTE: Este túnel se cerrará si detienes esta celda")
    print("="*70 + "\n")
    
    # Mostrar túneles activos
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print("📋 Túneles activos:")
        for t in tunnels:
            print(f"   • {t.public_url} -> {t.config['addr']}")
        print()
    
except Exception as e:
    print(f"❌ Error al crear túnel: {e}")
    print("\n💡 Soluciones posibles:")
    print("   1. Verifica tu token de ngrok")
    print("   2. Revisa tu conexión a internet")
    print("   3. Intenta ejecutar la celda nuevamente")
    raise

# Iniciar servidor Flask
print("🚀 Iniciando servidor Flask...\n")
print("="*70)
print("📝 LOGS DEL SERVIDOR (Ctrl+C para detener):")
print("="*70 + "\n")

try:
    # Ejecutar app.py
    !python app.py
    
except KeyboardInterrupt:
    print("\n\n⚠️  Servidor detenido por el usuario")
    ngrok.kill()
    print("✅ Túneles cerrados")
    
except Exception as e:
    print(f"\n\n❌ Error al ejecutar servidor: {e}")
    ngrok.kill()
    raise

finally:
    print("\n🔚 Limpieza completada")

In [ ]:
# @title 🔍 3. Diagnóstico y Pruebas (Opcional)

import os
import sys
from pathlib import Path

print("🔍 DIAGNÓSTICO DEL SISTEMA\n")
print("="*60)

# Información del sistema
print("\n📊 Sistema Operativo:")
!uname -a

print("\n🐍 Versión de Python:")
print(f"   {sys.version}")

print("\n📦 Paquetes principales instalados:")
packages = ["torch", "flask", "flask-socketio", "pyngrok", "opencv-python"]
for pkg in packages:
    try:
        result = !pip show {pkg} | grep Version
        if result:
            print(f"   ✅ {pkg}: {result[0].split(':')[1].strip()}")
        else:
            print(f"   ❌ {pkg}: No instalado")
    except:
        print(f"   ❌ {pkg}: Error al verificar")

print("\n🎮 Estado de GPU:")
try:
    import torch
    print(f"   GPU disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"   Nombre: {torch.cuda.get_device_name(0)}")
        print(f"   Memoria total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print(f"   Memoria libre: {torch.cuda.memory_reserved(0) / 1e9:.1f} GB")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n📁 Estructura del proyecto:")
backend_dir = Path("/content/FoadsIA/backend")
if backend_dir.exists():
    print(f"   📂 {backend_dir}")
    for item in sorted(backend_dir.iterdir())[:10]:
        icon = "📁" if item.is_dir() else "📄"
        print(f"      {icon} {item.name}")
    if len(list(backend_dir.iterdir())) > 10:
        print(f"      ... y {len(list(backend_dir.iterdir())) - 10} más")
else:
    print("   ❌ Directorio backend no encontrado")

print("\n" + "="*60)
print("✅ Diagnóstico completado")
print("="*60)

---

## 📚 Notas Importantes

### ⚡ Uso de GPU
- Asegúrate de seleccionar **GPU T4** en `Runtime > Change runtime type`
- La sesión de GPU es limitada en Colab gratuito (~12 horas)

### 🌐 Ngrok
- Obtén tu token gratuito en [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken)
- Sin token, las sesiones duran máximo 2 horas
- Con token, obtienes sesiones más largas y URLs personalizadas

### 🔒 Seguridad
- **NO compartas** tu token de ngrok públicamente
- La URL pública es temporal y expira cuando detienes el servidor
- Considera implementar autenticación en producción

### 💡 Troubleshooting
- Si el servidor no inicia, ejecuta la celda de diagnóstico
- Verifica que `app.py` exista en el repositorio
- Revisa los logs para identificar errores específicos

---

**Desarrollado por EnfoadsIA Team** 🚀